## Xarray engine: overview

Earthkit-data comes with its own Xarray engine called "earthkit" to perform conversions between GRIB and Xarray data.

To start with, we get the example data will use in this notebook and read it into a GRIB fieldlist.

In [1]:
import earthkit.data as ekd
ds_fl = ekd.from_source("sample", "pl.grib")

pl.grib:   0%|          | 0.00/48.8k [00:00<?, ?B/s]

### Creating Xarray

In [2]:
ds = ds_fl.to_xarray()
ds

From version 0.11.0 the default engine for to_xarray is 'earthkit'. Use engine=`cfgrib` to invoke the cfgrib engine.


<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, levelist: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * levelist                 (levelist) int64 16B 500 700
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, levelist, latitude, longitude) float64 88kB ...
Attributes:
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20240603
    time:         0
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

### Writing back to GRIB

This is an **experimental** feature!

In order to write back the Xarray into a GRIB it has to keep the original variable attributes that the eartkit engine generated. By default, variable attributes are not kept in Xarray computations so we need to set the global Xarray ``keep_attrs`` option to enable it as shown in the following cell: 

In [3]:
import xarray as xr
xr.set_options(keep_attrs=True)

ds = ds_fl.to_xarray()
ds += 1

#### Generating a fieldlist

In [4]:
ds_fl1 = ds.earthkit.to_fieldlist()
ds_fl1.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


We can see that the GRIB field values changed as expected if we compare the original and resulting fieldlists.

In [5]:
ds_fl.sel(param="t", step=6, level=500)[0].values.mean(), 
ds_fl1.sel(param="t", step=6, level=500)[0].values.mean()

255.25649845948692

#### Generating a GRIB file

In [6]:
ds_fl1.to_target("file", "_from_xr_1.grib")
ekd.from_source("file", "_from_xr_1.grib").head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


In [7]:
ds.earthkit.to_grib("_from_xr_2.grib")
ekd.from_source("file", "_from_xr_2.grib").head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll
